# Remapping of bed machine Antarctica 1

Coordinates is given in meters in a polar stereographic coordinate reference system. We need to create the corresponding longitude/latitude arrays using the PROJ library.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import cartopy.crs as ccrs
import xesmf

In [ ]:
#%matplotlib qt
%matplotlib inline

In [ ]:
_ = xr.set_options(display_style='text')

## Adding lon/lat from the CRS

bedmachine coordinates are carthesian (in $m$, centered on the south pole) in a polar stereographic projection. To get the corresponding longitude/latitude, we use the inverse transformation from the CRS:

The PROJ string is available here: 
    
https://nsidc.org/data/NSIDC-0756/versions/1

In [ ]:
PROJSTRING="+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

Write a function to compute the longitude and latitude using the inverse transformation from the CRS:

In [ ]:
def add_lon_lat(ds, PROJSTRING, x='x', y='y', chunks={}):
    """ add longitude and latitude as compute from the inverse projection
    given in PROJSTRING
    
    PARAMETERS:
    -----------
    ds: xarray.Dataset
    
    PROJSTRING: str
    
    """
    from pyproj import CRS, Transformer
    # create the coordinate reference system
    crs = CRS.from_proj4(PROJSTRING)
    # create the projection from lon/lat to x/y
    proj = Transformer.from_crs(crs.geodetic_crs, crs)
    # make x,y 2d arrays
    xx, yy = np.meshgrid(ds[x].values, ds[y].values)
    # compute the lon/lat
    lon, lat = proj.transform(xx, yy, direction='INVERSE')
    # add to dataset
    ds['lon'] = xr.DataArray(data=lon, dims=('y', 'x'))
    ds['lat'] = xr.DataArray(data=lat, dims=('y', 'x'))
    ds['lon'].attrs = dict(units='degrees_east')
    ds['lat'].attrs = dict(units='degrees_north')
    return ds

Fist time computing lon/lat takes about 2 minutes. Write the results into a new file. Next time, we can just load the file:

In [ ]:
compute_geo=False

In [ ]:
%%time

if compute_geo:
    bedmachine = xr.open_dataset('BedMachineAntarctica_2019-11-05_v01.nc')
    # this should take a couple of minutes
    bedmachine = add_lon_lat(bedmachine, PROJSTRING)
    bedmachine = bedmachine.assign_coords({'lon': bedmachine.lon,
                                           'lat': bedmachine.lat})
    bedmachine.to_netcdf('bedmachine+geo.nc', format='NETCDF3_64BIT')
else:
    bedmachine = xr.open_dataset('bedmachine+geo.nc')

In [ ]:
bedmachine

## Computing the elevation referenced to WGS84

[online doc](https://sites.uci.edu/morlighem/dataproducts/bedmachine-antarctica/) states: *All heights are referenced to mean sea level (using the geoid EIGEN-6C4). To convert the heights to heights referenced to the WGS84 ellipsoid, simply add the geoid height:*

$$ z_{ellipsoid}=z_{geoid}+geoid$$

which translates into:

In [ ]:
bedmachine['elevation'] = bedmachine['bed'] + bedmachine['geoid']

## Comparison with GEBCOv2

In [ ]:
gebco = xr.open_dataset('/archive/gold/datasets/topography/GEBCO_2020/GEBCO_2020.nc')

In [ ]:
gebco

For plotting, we don't need to use the full resolution of the datasets so we coarsen the datasets:

In [ ]:
gebco_topo_subsample = gebco.elevation.coarsen(lon=100, lat=100).mean()

In [ ]:
subplot_kws=dict(projection=ccrs.SouthPolarStereo(central_longitude=-120.),
                 facecolor='grey')

plt.figure(figsize=[12,8])
p = gebco_topo_subsample.plot(x='lon', y='lat',
                              vmin=-6000, vmax=6000,
                              cmap='bwr',
                              subplot_kws=subplot_kws,
                              transform=ccrs.PlateCarree(),
                              add_labels=False,
                              add_colorbar=True)

p.axes.gridlines(color='black', alpha=0.5, linestyle='--')
p.axes.set_extent([-300, 60, -60, -90], ccrs.PlateCarree())
p.axes.set_title('GEBCO2')

In [ ]:
# round up for coarsening
start=0
end=13000

bedtmp = bedmachine.isel(x=slice(start,end),
                         y=slice(start,end))
                         

bedmachine_bed_subsample = bedtmp.elevation.coarsen(x=50, y=50).mean()


bedmachine_bed_subsample['lon'] = xr.DataArray(bedtmp['lon'].values[::50,::50],
                                               dims=('y', 'x'))
bedmachine_bed_subsample['lat'] = xr.DataArray(bedtmp['lat'].values[::50,::50],
                                               dims=('y', 'x'))

subplot_kws=dict(projection=ccrs.SouthPolarStereo(central_longitude=-120.),
                 facecolor='grey')

plt.figure(figsize=[12,8])
p = bedmachine_bed_subsample.plot(x='lon', y='lat',
                                  vmin=-6000, vmax=6000,
                                  cmap='bwr',
                                  subplot_kws=subplot_kws,
                                  transform=ccrs.PlateCarree(),
                                  add_labels=False,
                                  add_colorbar=True)

p.axes.gridlines(color='black', alpha=0.5, linestyle='--')
p.axes.set_extent([-300, 60, -60, -90], ccrs.PlateCarree())
p.axes.set_title('Bedmachine bedrock, ref to WGS84')

## Remapping onto GEBCOv2 grid:

Because of the bedmachine projection geometry, we can only remap bedmachine south of 62S.

To be computationally efficient, we subset the coordinates of the GEBCO grid south of 62S:

In [ ]:
gebco_SO = gebco.sel(lat=slice(-90, -62))

In [ ]:
gebco_SO['lon'].attrs = dict(units='degrees_east')
gebco_SO['lat'].attrs = dict(units='degrees_north')

The regridding is too big to be done in a reasonable time without a HPC system. On a HPC with ESMF installed (e.g. using conda + install ESMF/ESMPy packages), we can use the parallel weights generation tool and an example invocation is:

the weight generation has its own [github](https://github.com/raphaeldussin/regrid_weights_bedmachine_gebco)/[gitlab](https://gitlab.gfdl.noaa.gov/ogrp/regrid_weights_bedmachine_gebco) repos and results were stored on PPAN:

In [ ]:
weightsdir = '/work/Raphael.Dussin/weights_save/bedmachine_gebco_gaea'

Once the weights are created, we can use them within xESMF:

In [ ]:
%%time

regrid = xesmf.Regridder(bedmachine, gebco_SO,
                         method='nearest_s2d',
                         periodic=False,
                         reuse_weights=True,
                         filename=f'{weightsdir}/nearest_weights_srcR.nc')

And create the remapped elevation:

In [ ]:
%%time

bedmachine_remapped = xr.Dataset()

bedmachine_remapped['elevation'] = regrid(bedmachine['elevation'])

# gebco is given in type short, which basically is equivalent to integer values,
# hence no need to save decimal points...
encoding = {'elevation': {'dtype': np.int16}}
bedmachine_remapped.to_netcdf('BedMachineAntarctica_elevation_nn_gebco2020.nc',
                              encoding=encoding, format='NETCDF3_64BIT',
                              engine='netcdf4')

In [ ]:
%%time

remap_extras=False

if remap_extras:
    bedmachine_remapped_extras = xr.Dataset()
    for var in ['bed', 'firn', 'surface', 'thickness', 'geoid']:
        bedmachine_remapped_extras[var] = regrid(bedmachine[var])
    bedmachine_remapped_extras.to_netcdf('BedMachineAntarctica_regridded_nn_gebco2020.nc')

In [ ]:
plt.figure(figsize=[12,8])
bedmachine_remapped['elevation'].plot()

The resulting remapping passes the eyeball test. To be certain we can plot the difference with gebco:

In [ ]:
plt.figure(figsize=[12,8])
diffbathy = bedmachine_remapped['elevation'].astype(np.int16) - gebco_SO['elevation']
diffbathy.plot()

This shows clearly the difference between bedmachine bedrock and gebco "top of the ice" elevation. Focusing on the northern part of the domain, where the merging will happen, we don't notice any suspicious mismatch (just noise)

In [ ]:
plt.figure(figsize=[12,8])
diffbathy.sel(lat=slice(-65,-62)).plot(vmin=-100, vmax=100, cmap='bwr')

Continue to merge bathy...